<a href="https://colab.research.google.com/github/Sreemaee21/SpeechProcessing/blob/main/Copy_of_BagOfWords_Ohsumed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets



In [ ]:
import pickle
from datasets import load_dataset
import pandas as pd


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
import pickle
from tqdm.notebook import tqdm




In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

uncomment the below cell to run complete code from scratch

expected output:



Downloading builder script: 100%
10.1k/10.1k [00:00<00:00, 123kB/s]
Downloading metadata: 100%
2.73k/2.73k [00:00<00:00, 50.1kB/s]
Downloading readme: 100%
7.95k/7.95k [00:00<00:00, 145kB/s]
Downloading data: 100%
139M/139M [00:02<00:00, 60.2MB/s]
Generating train split: 100%
54709/54709 [00:09<00:00, 10156.47 examples/s]
Generating test split: 100%
293855/293855 [00:35<00:00, 5728.47 examples/s]

In [ ]:


# Load the ohsumed dataset
dataset = load_dataset('ohsumed')


# Split the dataset
train_data = dataset["train"]
test_data = dataset["test"]


<!-- uncomment the below cell to run complete code from scratch -->

expected output:


Missing values in 'train' split:
seq_id              0
medline_ui          0
mesh_terms          0
title               0
publication_type    0
abstract            0
author              0
source              0
dtype: int64

Missing values in 'test' split:
seq_id              0
medline_ui          0
mesh_terms          0
title               0
publication_type    0
abstract            0
author              0
source              0
dtype: int64

In [ ]:


missing_values_train = dataset['train'].data.to_pandas().isnull().sum()
print("Missing values in 'train' split:")
print(missing_values_train)

# Check for missing values in the 'test' split
missing_values_test = dataset['test'].data.to_pandas().isnull().sum()
print("\nMissing values in 'test' split:")
print(missing_values_test)


Missing values in 'train' split:
seq_id              0
medline_ui          0
mesh_terms          0
title               0
publication_type    0
abstract            0
author              0
source              0
dtype: int64

Missing values in 'test' split:
seq_id              0
medline_ui          0
mesh_terms          0
title               0
publication_type    0
abstract            0
author              0
source              0
dtype: int64


uncomment the below cell to run complete code from scratch


expected output:

dict_keys(['train', 'test'])

In [ ]:

# print(dataset.keys())





uncomment the below cell to run complete code from scratch

expected output:



Train dataset:
Dataset({
    features: ['seq_id', 'medline_ui', 'mesh_terms', 'title', 'publication_type', 'abstract', 'author', 'source'],
    num_rows: 54709
})

Test dataset:
Dataset({
    features: ['seq_id', 'medline_ui', 'mesh_terms', 'title', 'publication_type', 'abstract', 'author', 'source'],
    num_rows: 293855
})


In [ ]:

# View the first few rows of each dataset
print("Train dataset:")
print(train_data)
train_data_df = pd.DataFrame(dataset['train'])

print("\nTest dataset:")
print(test_data)
test_data_df = pd.DataFrame(dataset['test'])



Train dataset:
Dataset({
    features: ['seq_id', 'medline_ui', 'mesh_terms', 'title', 'publication_type', 'abstract', 'author', 'source'],
    num_rows: 54709
})

Test dataset:
Dataset({
    features: ['seq_id', 'medline_ui', 'mesh_terms', 'title', 'publication_type', 'abstract', 'author', 'source'],
    num_rows: 293855
})


## **Pre-Processing**

In [ ]:


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# # Load the ohsumed dataset
# dataset = load_dataset('ohsumed')

# Load the stopwords
stop_words = set(stopwords.words('english'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

# Define the preprocessing function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text



# **Train data prep**

Storing all pre-processed train data in pickle files


uncomment the below cell to run complete code from scratch

expected output:

100%
54709/54709 [03:29<00:00, 127.12it/s]

In [ ]:


# Create a list to store processed data
processed_data_list = []

# Loop through train_data and preprocess
for data in tqdm(train_data):
    # Preprocess 'title', 'abstract', and 'mesh_terms' fields
    data['title'] = preprocess_text(data['title'])
    data['abstract'] = preprocess_text(data['abstract'])
    data['mesh_terms'] = preprocess_text(data['mesh_terms'])

    # Append the processed data dictionary to the list
    processed_data_list.append(data)

# Save the processed_data_list to a pickle file
with open('processed_data.pickle', 'wb') as f:
    pickle.dump(processed_data_list, f)


  0%|          | 0/54709 [00:00<?, ?it/s]

Loading train data from pickle file


In [ ]:


# Load the processed data from the pickle file
with open('processed_data.pickle', 'rb') as f:
    loaded_processed_data = pickle.load(f)

# Now you can access the processed data dictionaries in the loaded_processed_data list


In [ ]:

# Load the processed_data_list from the pickle file
with open('processed_data.pickle', 'rb') as f:
    processed_data_list = pickle.load(f)


# Create a list to store preprocessed mesh terms
preprocessed_mesh_terms = [data['mesh_terms'] for data in processed_data_list]

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(preprocessed_mesh_terms)

# Now X_train_bow will contain the Bag-of-Words representation of the training data


In [ ]:
print(X_train_bow)


  (0, 375)	1
  (0, 4075)	1
  (0, 6910)	1
  (0, 2980)	1
  (0, 2177)	1
  (0, 3027)	2
  (0, 5538)	1
  (0, 8907)	1
  (0, 4304)	1
  (0, 7488)	1
  (0, 7835)	1
  (0, 8780)	1
  (0, 3953)	1
  (0, 9127)	1
  (0, 3395)	1
  (0, 9250)	1
  (0, 6772)	1
  (0, 9610)	1
  (0, 3482)	1
  (0, 8999)	1
  (1, 3027)	2
  (1, 4304)	1
  (1, 7488)	1
  (1, 8999)	1
  (1, 623)	1
  :	:
  (54708, 502)	2
  (54708, 2275)	1
  (54708, 6929)	2
  (54708, 6632)	1
  (54708, 3570)	1
  (54708, 6779)	1
  (54708, 8793)	1
  (54708, 6286)	1
  (54708, 629)	1
  (54708, 4544)	3
  (54708, 3575)	1
  (54708, 4746)	1
  (54708, 9410)	1
  (54708, 1574)	1
  (54708, 780)	1
  (54708, 5872)	1
  (54708, 8839)	1
  (54708, 2347)	1
  (54708, 6971)	1
  (54708, 4528)	1
  (54708, 631)	1
  (54708, 5335)	1
  (54708, 9233)	1
  (54708, 4251)	1
  (54708, 2825)	1


In [ ]:
print(X_train_bow.shape)



(54709, 9876)


In [ ]:

# Create a DataFrame from the processed data
train_data_df = pd.DataFrame(processed_data_list)

# Combine 'title' and 'abstract' columns into 'combined_text'
train_data_df['combined_text'] = train_data_df['title'].apply(' '.join) + ' ' + train_data_df['abstract'].apply(' '.join)


# Check if there are any empty or very short documents
short_documents = train_data_df[train_data_df['combined_text'].str.split().str.len() <= 3]
print(short_documents)


Empty DataFrame
Columns: [seq_id, medline_ui, mesh_terms, title, publication_type, abstract, author, source, combined_text]
Index: []


In [ ]:

# Check for missing values
missing_values = train_data_df.isnull().sum()
print(missing_values)

seq_id              0
medline_ui          0
mesh_terms          0
title               0
publication_type    0
abstract            0
author              0
source              0
combined_text       0
dtype: int64


In [ ]:
train_data_df


,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source,combined_text
0,1,87049087,allied health personnel/ electric countershock...,refibrillation managed emt-ds incidence outcom...,JOURNAL ARTICLE.,patient converted ventricular fibrillation org...,Stults KR; Brown DD.,Am J Emerg Med 8703; 4(6):491-5,r e f i b r i l l a t i o n m a n a g e d ...
1,1,87049088,antidepressive agent tricyclic/ po arrhythmia/...,tricyclic antidepressant overdose emergency de...,JOURNAL ARTICLE.,controversy regarding appropriate utilization ...,Foulke GE; Albertson TE; Walby WF.,Am J Emerg Med 8703; 4(6):496-500,t r i c y c l i c a n t i d e p r e s s a n ...
2,1,87049089,adult aircraft/ altitude/ blood gas monitoring...,transconjunctival oxygen monitoring predictor ...,JOURNAL ARTICLE.,use helicopter air transport critically ill pa...,Shufflebarger C; Jehle D; Cottington E; Martin M.,Am J Emerg Med 8703; 4(6):501-3,t r a n s c o n j u n c t i v a l o x y g e ...
3,1,87049090,adolescence adult aged blood glucose/ diabetes...,serum glucose change administration 50 dextros...,JOURNAL ARTICLE.,prospective clinical trial conducted estimate ...,Adler PM.,Am J Emerg Med 8703; 4(6):504-6,s e r u m g l u c o s e c h a n g e a d ...
4,1,87049092,aged aged 80 case report female human intubati...,nasogastric intubation morbidity asymptomatic ...,JOURNAL ARTICLE.,unusual case misdirected nasogastric tube desc...,Gough D; Rust D.,Am J Emerg Med 8703; 4(6):511-3,n a s o g a s t r i c i n t u b a t i o n ...
...,...,...,...,...,...,...,...,...,...
54704,1,87325763,autoantibodies/ blood coagulation factors/an/ ...,spectrum neurologic disease associated antipho...,JOURNAL ARTICLE; REVIEW.,lupus anticoagulant anticardiolipin antibody a...,Levine SR; Welch KM.,Arch Neurol 8712; 44(8):876-83,s p e c t r u m n e u r o l o g i c d i s ...
54705,1,87325791,animal arachidonic acids/me biomechanics endop...,prostaglandin old concept new perspective,JOURNAL ARTICLE.,lupus anticoagulant anticardiolipin antibody a...,Bito LZ.,Arch Ophthalmol 8712; 105(8):1036-9,p r o s t a g l a n d i n o l d c o n c e ...
54706,1,87325811,actins/me animal cell adhesion cell movement c...,vinculin focal cell-to-substrate attachment sp...,JOURNAL ARTICLE.,vinculin specialized protein molecular weight ...,Soong HK.,Arch Ophthalmol 8712; 105(8):1129-32,v i n c u l i n f o c a l c e l l - t o - ...
54707,1,87325849,alteplase/ pd animal cornea/ah dose-response r...,intraocular fibrinolysis recombinant human tis...,JOURNAL ARTICLE.,intraocular fibrin formation following ocular ...,Snyder RW; Lambrou FH; Williams GA.,Arch Ophthalmol 8712; 105(9):1277-80,i n t r a o c u l a r f i b r i n o l y s i ...


In [ ]:

# Create a DataFrame from the processed data
train_data_df = pd.DataFrame(processed_data_list)

# Combine 'title' and 'abstract' columns into 'combined_text'
train_data_df['combined_text'] = train_data_df['title'] + ' ' + train_data_df['abstract']

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(train_data_df['combined_text'])

# Save the fitted vectorizer using pickle
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)


In [ ]:

# Initialize the CountVectorizer again (not necessary if you've already fitted it)
vectorizer = CountVectorizer()

# Fit and transform the training data again (not necessary if you've already done it)
X_train_bow = vectorizer.fit_transform(train_data_df['combined_text'])

# Get the vocabulary (feature names) from the CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Print the first 70 feature names
print(feature_names[:70])


['00' '000' '0000' '000001' '000002' '00001' '00002' '00003' '000037'
 '00005' '00006' '00008' '0001' '000148' '0002' '0003' '00038' '0004'
 '0005' '0006' '0007' '0008' '00085' '00087' '0009' '00097' '000pc' '001'
 '0010' '0011' '0012' '0013' '0014' '0015' '0016' '0017' '0018' '0019'
 '002' '0020' '0022' '0023' '0024' '0025' '0026' '0027' '0029' '003'
 '0030' '0030x' '0032' '00321' '0033' '00335' '0034' '0035' '0036' '0038'
 '0039' '003m' '004' '0041' '0042' '0043' '0045' '0046' '0047' '0048'
 '0049' '005']


# **test data**


uncomment the below cell to run complete code from scratch
The preprocessed test data will be stored in a

expected output:


100%
293855/293855 [15:30<00:00, 311.54it/s]




In [ ]:


# Create a list to store preprocessed test data
processed_test_data_list = []

# Loop through test_data and preprocess
for data in tqdm(test_data):
    # Preprocess 'title', 'abstract', and 'mesh_terms' fields
    data['title'] = preprocess_text(data['title'])
    data['abstract'] = preprocess_text(data['abstract'])
    data['mesh_terms'] = preprocess_text(data['mesh_terms'])

    # Append the preprocessed data dictionary to the list
    processed_test_data_list.append(data)

# Save the processed_test_data_list to a pickle file
with open('processed_test_data.pickle', 'wb') as f:
    pickle.dump(processed_test_data_list, f)


  0%|          | 0/293855 [00:00<?, ?it/s]

In [ ]:

# #Create a DataFrame from the processed test data

In [ ]:
with open('processed_test_data.pickle', 'rb') as f:
    vectorizer = pickle.load(f)


test_data_df = pd.DataFrame(processed_test_data)

# # Combine 'title' and 'abstract' columns into 'combined_text'
# test_data_df['combined_text'] = test_data_df['title'] + ' ' + test_data_df['abstract']

# # Load the vectorizer from the training data (assuming 'vectorizer' was saved)
# Load the fitted vectorizer from the pickle file
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)

# Preprocessing for the test data (assuming you've loaded and preprocessed it)
test_data_df['combined_text'] = test_data_df['title'] + ' ' + test_data_df['abstract']

# Transform the test data using the loaded vectorizer
X_test_bow = vectorizer.transform(test_data_df['combined_text'])


NameError: ignored

In [ ]:
#CHUNK 1
# Create a DataFrame from the processed test data
test_data_df = pd.DataFrame(processed_test_data_list)

# Combine 'title' and 'abstract' columns into 'combined_text'
test_data_df['combined_text'] = test_data_df['title'] + ' ' + test_data_df['abstract']

# Load the vectorizer from the training data (assuming 'vectorizer' was saved)
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)


In [ ]:
#chunk 2

# Initialize a batch size for processing
batch_size = 1000

# Initialize the StandardScaler
scaler = StandardScaler()

# Initialize an empty list to store scaled batches
scaled_batches = []

# Loop through batches and perform scaling
for batch_start in range(0, len(test_data_df), batch_size):
    batch_end = batch_start + batch_size
    batch = test_data_df.iloc[batch_start:batch_end]

    # Vectorization (use the same vectorizer from the training data)
    batch_vectorized = vectorizer.transform(batch['combined_text'])

    # Feature Scaling (use the same scaler from the training data)
    batch_scaled = scaler.fit_transform(batch_vectorized.toarray())
    scaled_batches.append(batch_scaled)

In [ ]:


# import dask.dataframe as dd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import StandardScaler
# import pickle

# # Load the processed data from the pickle file
# with open('processed_data.pickle', 'rb') as f:
#     processed_data_list = pickle.load(f)

# # Convert the processed data list to a pandas DataFrame
# import pandas as pd
# train_data_df = pd.DataFrame(processed_data_list)

# # Combine 'title' and 'abstract' columns into 'combined_text'
# train_data_df['combined_text'] = train_data_df['title'] + ' ' + train_data_df['abstract']

# # Initialize the TfidfVectorizer
# vectorizer = TfidfVectorizer()

# # Fit and transform the training data
# X_train_tfidf = vectorizer.fit_transform(train_data_df['combined_text'])

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Transform the TF-IDF vectors using the same scaler
# X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())

# # Save the fitted vectorizer for later use
# with open('tfidf_vectorizer.pickle', 'wb') as f:
#     pickle.dump(vectorizer, f)


In [ ]:
# Create a DataFrame from the processed test data
test_data_df = pd.DataFrame(processed_test_data_list)

# Combine 'title' and 'abstract' columns into 'combined_text'
test_data_df['combined_text'] = test_data_df['title'] + ' ' + test_data_df['abstract']

# Load the vectorizer from the training data (assuming 'vectorizer' was saved)
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)


In [ ]:

# Load the SVM model (assuming 'svm_model' was trained and saved)
with open('svm_model.pickle', 'rb') as f:
    svm_model = pickle.load(f)


In [ ]:

# # Initialize a batch size for processing
# batch_size = 1000

# # Initialize an empty list to store predictions
# all_test_predictions = []

# # Loop through batches and perform predictions
# for batch_start in range(0, len(test_data_df), batch_size):
#     batch_end = batch_start + batch_size
#     batch = test_data_df.iloc[batch_start:batch_end]

#     # Vectorization (use the same vectorizer from the training data)
#     batch_vectorized = vectorizer.transform(batch['combined_text'])

#     # SVM Prediction
#     batch_predictions = svm_model.predict(batch_vectorized)
#     all_test_predictions.extend(batch_predictions)

# # Print or use the test_predictions for further analysis
# print("Test predictions:", all_test_predictions)




import dask.dataframe as dd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import pickle

# Load the processed data from the pickle file
with open('processed_data.pickle', 'rb') as f:
    processed_data_list = pickle.load(f)

# Create a Dask DataFrame from the processed data
ddf = dd.from_pandas(processed_data_list, npartitions=10)  # You can adjust the number of partitions

# Combine 'title' and 'abstract' columns into 'combined_text'
ddf['combined_text'] = ddf['title'] + ' ' + ddf['abstract']

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Initialize the StandardScaler
scaler = StandardScaler()

# Initialize an empty list to store scaled batches
scaled_batches = []

# Loop through partitions and perform scaling
for partition in ddf.partitions:

    # Vectorization (use the same vectorizer from the training data)
    partition_vectorized = vectorizer.transform(partition['combined_text'])

    # Feature Scaling (use the same scaler from the training data)
    partition_scaled = scaler.fit_transform(partition_vectorized.toarray())
    scaled_batches.append(partition_scaled)

# Concatenate the scaled batches
X_test_bow = dd.concatenate(scaled_batches)

# Compute the result to get a NumPy array
X_test_bow = X_test_bow.compute()

# Now you can use X_test_bow for predictions



In [ ]:
#oracle test assesment
#https://www.hackerrank.com/test/86tthog809l/login?b=eyJ1c2VybmFtZSI6InNyZWVtYWVlMjEyMUBnbWFpbC5jb20iLCJwYXNzd29yZCI6ImFmMDMzMzJhIiwiaGlkZSI6dHJ1ZSwiYWNjb21tb2RhdGlvbnMiOnsiYWRkaXRpb25hbF90aW1lX3BlcmNlbnQiOjB9fQ==